In [1]:
import numpy as np

In [6]:
a = np.array([
    [1, 2, 3],
])

b = np.array([
    [4, 5, 6],
])

c = np.empty(6, dtype = int)

c[0::2] = a
c[1::2] = b
c

array([1, 4, 2, 5, 3, 6])

In [3]:
for i in a:
    print(i)

[1 2 3]
[4 3 2]


In [4]:
a.shape

(2, 3)

In [49]:
np.tile(
    b,
    2
)

array([3, 6, 2, 1, 5, 4, 7, 8, 3, 6, 2, 1, 5, 4, 7, 8])

In [2]:
from torch.utils.data import Sampler

In [3]:
from typing import List

In [4]:
np.concatenate([np.full(10, 0), np.full(6, 1), np.empty(0)])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.])

In [18]:
a = {
    'd' : 1,
    'b' : 2,
    'a' : 3,
    'z' : 5,
    'y' : 20,
    'n' : 200
}

In [21]:
for idx_key, key in enumerate(sorted(a)):
    print(key, idx_key)

a 0
b 1
d 2
n 3
y 4
z 5


In [10]:
b = {
    label: np.arange(len(labels))[labels == label].tolist()
    for label in set(labels)
    }

In [43]:
b = np.array([
    [1, 2, 3, 4,],
    [5, 6, 7, 8],
])
b = b.flatten()

In [44]:
np.random.shuffle(b)

In [45]:
b

array([3, 6, 2, 1, 5, 4, 7, 8])

In [53]:
def class_labels(ratios):
    # self.ratios keys are in insertion order for python ge 3.7
    labels = np.empty(sum(ratios.values())).astype(int)
    # will be like [0, 0, 0, 1, 1, 1, 1, 1, ...]
    start = 0
    for idx_contrast, contrast_count in enumerate(ratios.values()):
        labels[start : start + contrast_count] = np.full(
            contrast_count, idx_contrast
            )
        start += contrast_count
    return labels

In [54]:
class_labels(a)

array([0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [51]:
def class_labels(ratios):
    # self.ratios keys are in insertion order for python ge 3.7
    labels = []
    # will be like [0, 0, 0, 1, 1, 1, 1, 1, ...]
    for idx_contrast, contrast_count in enumerate(ratios.values()):
        labels.append(
            np.full(contrast_count, idx_contrast)
            )
    labels = np.concatenate(labels)
    return labels

In [25]:
for idx, value in enumerate(a.values()):
    print(idx, value)

0 1
1 2
2 3
3 5
4 20
5 200


In [3]:
import bokeh.plotting
bokeh.io.output_notebook()
import csv
import pandas as pd
from pathlib import Path
import os
import numpy as np
from bokeh.palettes import Colorblind7

Loading BokehJS ...

In [4]:
Colorblind7

('#0072B2', '#E69F00', '#F0E442', '#009E73', '#56B4E9', '#D55E00', '#CC79A7')

In [220]:
path1 = os.path.join(
    'models/MTL_naive_varnet0:2_div_coronal_pd_fs_div_coronal_pd', 
    'summary_div_coronal_pd_fs.csv'
)

path2 = os.path.join(
    'models/MTL_naive_varnet0:2_div_coronal_pd_fs_div_coronal_pd',
    'summary_div_coronal_pd.csv'
)
summary_file1 = Path(path1)
summary_file2 = Path(path2)

In [221]:
df1 = pd.read_csv(summary_file1)
df2 = pd.read_csv(summary_file2)

In [218]:
df1 = df1.sort_values(by=[f'div_coronal_pd_fs'])
df2 = df2.sort_values(by=[f'div_coronal_pd_fs'])
df1 = df1.drop('Unnamed: 0', axis = 1)
df2 = df2.drop('Unnamed: 0', axis = 1)
df2 = df2.drop(0)
df2 = df2.drop(['loss', 'nrmse'], axis = 1)
l = [df1, df2]
df1, df2

(       loss      ssim       psnr     nrmse  div_coronal_pd_fs  div_coronal_pd
 0  0.039891  0.812367  31.642353  0.157984           0.064386             1.0
 2  0.033093  0.848451  33.234558  0.131297           0.215292             1.0
 1  0.032012  0.855036  33.682333  0.124817           0.509054             1.0
 3  0.031127  0.860630  33.996669  0.120426           1.000000             1.0,
        ssim       psnr  div_coronal_pd_fs  div_coronal_pd
 2  0.898562  34.133462           0.215292             1.0
 1  0.902340  34.374319           0.509054             1.0
 3  0.902684  34.348572           1.000000             1.0)

In [214]:
names_min = pd.DataFrame(
    np.zeros([4, 4]), 
    columns = [
        'loss', 'ssim', 
        'psnr', 'nrmse', 
    ])

names_max = pd.DataFrame(
    np.zeros([4, 4]), 
    columns = [
        'loss', 'ssim', 
        'psnr', 'nrmse', 
    ])
names_min, names_max

(   loss  ssim  psnr  nrmse
 0   0.0   0.0   0.0    0.0
 1   0.0   0.0   0.0    0.0
 2   0.0   0.0   0.0    0.0
 3   0.0   0.0   0.0    0.0,
    loss  ssim  psnr  nrmse
 0   0.0   0.0   0.0    0.0
 1   0.0   0.0   0.0    0.0
 2   0.0   0.0   0.0    0.0
 3   0.0   0.0   0.0    0.0)

In [207]:
l

[       loss      ssim       psnr     nrmse  div_coronal_pd_fs  div_coronal_pd
 0  0.039891  0.812367  31.642353  0.157984           0.064386             1.0
 2  0.033093  0.848451  33.234558  0.131297           0.215292             1.0
 1  0.032012  0.855036  33.682333  0.124817           0.509054             1.0
 3  0.031127  0.860630  33.996669  0.120426           1.000000             1.0,
        ssim       psnr  div_coronal_pd_fs  div_coronal_pd
 2  0.898562  34.133462           0.215292             1.0
 1  0.902340  34.374319           0.509054             1.0
 3  0.902684  34.348572           1.000000             1.0]

In [212]:
DF_psnr_ssim = pd.concat(l, keys=range(len(l)))
DF_psnr_ssim

loss      ssim       psnr     nrmse  div_coronal_pd_fs  \
0 0  0.039891  0.812367  31.642353  0.157984           0.064386   
  2  0.033093  0.848451  33.234558  0.131297           0.215292   
  1  0.032012  0.855036  33.682333  0.124817           0.509054   
  3  0.031127  0.860630  33.996669  0.120426           1.000000   
1 2       NaN  0.898562  34.133462       NaN           0.215292   
  1       NaN  0.902340  34.374319       NaN           0.509054   
  3       NaN  0.902684  34.348572       NaN           1.000000   

     div_coronal_pd  
0 0             1.0  
  2             1.0  
  1             1.0  
  3             1.0  
1 2             1.0  
  1             1.0  
  3             1.0

In [213]:
DF_psnr_ssim = pd.concat(l, keys=range(len(l))).drop(['loss', 'nrmse'], axis = 1).groupby(level=1)
DF_loss_nrmse = pd.concat(l, keys=range(len(l))).drop(['psnr', 'ssim'], axis = 1).groupby(level=1)

maxdf = DF_psnr_ssim.max()
mindf = DF_loss_nrmse.min()

maxdf

,ssim,psnr,div_coronal_pd_fs,div_coronal_pd
0,0.812367,31.642353,0.064386,1.0
1,0.902340,34.374319,0.509054,1.0
2,0.898562,34.133462,0.215292,1.0
3,0.902684,34.348572,1.000000,1.0


In [175]:
mindf

,loss,ssim,psnr,nrmse,div_coronal_pd_fs,div_coronal_pd
0,0.039891,0.812367,31.642353,0.157984,0.064386,1.0
1,0.019098,0.855036,33.682333,0.077433,0.509054,1.0
2,0.019480,0.848451,33.234558,0.079579,0.215292,1.0
3,0.019164,0.860630,33.996669,0.077470,1.000000,1.0


In [252]:
names_max[maxdf.eq(df2)] = 'div_coronal_pd'

In [253]:
names_max[maxdf.eq(df1)] = 'div_coronal_pd_fs'

In [254]:
names_max = names_max.rename(
    columns = {
    'loss' : 'lossname',
    'ssim' : 'ssimname',
    'psnr' : 'psnrname',
    'nrmse' : 'nrmsename',
}
)

In [255]:
# maybe none of the runs have certain scarcities
names_max = names_max[(names_max.T != 0).any()]
names_max

,lossname,ssimname,psnrname,nrmsename


In [256]:
ssim_psnr = pd.concat([maxdf, zeros], axis = 1)
ssim_psnr

,ssim,psnr,div_coronal_pd_fs,div_coronal_pd,lossname,ssimname,psnrname,nrmsename
0,0.812367,31.642353,0.064386,1.0,div_coronal_pd_fs,div_coronal_pd_fs,div_coronal_pd_fs,div_coronal_pd_fs
1,0.902340,34.374319,0.509054,1.0,div_coronal_pd_fs,div_coronal_pd,div_coronal_pd,div_coronal_pd_fs
2,0.898562,34.133462,0.215292,1.0,div_coronal_pd_fs,div_coronal_pd,div_coronal_pd,div_coronal_pd_fs
3,0.902684,34.348572,1.000000,1.0,div_coronal_pd_fs,div_coronal_pd,div_coronal_pd,div_coronal_pd_fs


In [224]:
a = df1.loc[df1['div_coronal_pd_fs'] == max(df1['div_coronal_pd_fs']), 'psnr']
float(a)

33.99666931421489

In [248]:
p = bokeh.plotting.figure(
            width = 700,
            height = 340,
            x_axis_label = 'percent',
            y_axis_label = 'psnr',
)

p.square(
    source = df1,
    x = 'div_coronal_pd_fs',
    y = 'psnr',
    legend_label = f'STL baseline div_coronal_pd',
    color="orange",fill_color=None, line_width=2, size = 15
)

bokeh.io.show(p)

In [280]:
if True:
    pass
elif True:
    print('hi')